In [ ]:
%pip install -q -U "google-genai>=1.43.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 8.5 MB/s eta 0:00:00


## Gemini File Search Tool API: Using Custom chunking and Meta data

<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/file_search/gemini_file_search_tool_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

put the instructions in here


In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# !wget https://storage.courtlistener.com/recap/gov.uscourts.cand.433688/gov.uscourts.cand.433688.340.1.pdf -O document.pdf

In [ ]:
!unzip -q /content/transcripts.zip

Archive:  /content/transcripts.zip
   creating: transcripts/
  inflating: transcripts/github_universe.md  
  inflating: transcripts/claude_skills.md  
  inflating: transcripts/langchain_1.0.md  
  inflating: transcripts/openai_atlas_browser.md  
  inflating: transcripts/is_meta_killing_fair.md  
  inflating: transcripts/kimi_k2_thinking.md  
  inflating: transcripts/haiku_4.5.md  
  inflating: transcripts/deepseek_ocr.md  


In [ ]:
from google import genai
from google.genai import types
import time

In [ ]:
client = genai.Client()

# Create the file search store with an optional display name
file_search_store = client.file_search_stores.create(config={'display_name': 'sam-youtube-transcripts'})

In [ ]:
file_search_store

FileSearchStore(
  create_time=datetime.datetime(2025, 11, 9, 11, 5, 3, 395867, tzinfo=TzInfo(UTC)),
  display_name='sam-youtube-transcripts',
  name='fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0',
  update_time=datetime.datetime(2025, 11, 9, 11, 5, 3, 395867, tzinfo=TzInfo(UTC))
)

In [ ]:
# Create a file search store (including optional display_name for easier reference)
# file_search_store = client.file_search_stores.create(config={'display_name': 'my-file_search-store-123'})

# List all your file search stores
for file_search_store in client.file_search_stores.list():
    print(file_search_store)

name='fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0' display_name='sam-youtube-transcripts' create_time=datetime.datetime(2025, 11, 9, 11, 5, 3, 395867, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 11, 9, 11, 5, 3, 395867, tzinfo=TzInfo(UTC)) active_documents_count=None pending_documents_count=None failed_documents_count=None size_bytes=None


### Set the Chunking for uploading

In [ ]:
chunking_config = {
    'chunking_config': {
        'white_space_config': {
            'max_tokens_per_chunk': 250,
            'max_overlap_tokens': 50
            }
        }
    }

### Upload multiple files

In [ ]:
import glob

transcript_files = glob.glob('/content/transcripts/*')
print(transcript_files)


['/content/transcripts/deepseek_ocr.md', '/content/transcripts/is_meta_killing_fair.md', '/content/transcripts/langchain_1.0.md', '/content/transcripts/claude_skills.md', '/content/transcripts/kimi_k2_thinking.md', '/content/transcripts/openai_atlas_browser.md', '/content/transcripts/github_universe.md', '/content/transcripts/haiku_4.5.md']


In [ ]:
import os

def extract_title_and_url(file_path):
    """
    Extracts the title and URL from the first two lines of a file.

    Args:
        file_path (str): The path to the file.

    Returns:
        tuple: A tuple containing the title and URL (title, url).
               Returns (None, None) if the file cannot be read or doesn't
               have at least two lines.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            if len(lines) >= 2:
                title = lines[0].strip().lstrip('# ')
                url = lines[1].strip().lstrip('URL: ')
                return title, url
            else:
                return None, None
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None, None

In [ ]:
import os

file_path = '/content/transcripts/deepseek_ocr.md'
file_name = os.path.basename(file_path)
print(file_name)

deepseek_ocr.md


In [ ]:
for file_path in transcript_files:
    title, url = extract_title_and_url(file_path )
    file_name = os.path.basename(file_path)
    print("-"*20)
    print(f"title: {title}")
    print(f"url: {url}")
    print(f"file_name: {file_name}")

--------------------
title: (343) DeepSeek OCR - More than OCR - YouTube
url: https://www.youtube.com/watch?v=YEZHU4LSUfU
file_name: deepseek_ocr.md
--------------------
title: (343) Is Meta killing FAIR? - YouTube
url: https://www.youtube.com/watch?v=zda-H480-vY
file_name: is_meta_killing_fair.md
--------------------
title: (343) LangChain Reaches 1.0 - Whats new? - YouTube
url: https://www.youtube.com/watch?v=hjIlj4OAECQ
file_name: langchain_1.0.md
--------------------
title: (343) Claude Skills - SOPs For Agents - YouTube
url: https://www.youtube.com/watch?v=fvUGQFtJaT4
file_name: claude_skills.md
--------------------
title: (343) NEW Kimi K2 Thinking - Think Harder - YouTube
url: https://www.youtube.com/watch?v=rX_UwjUt_4Q
file_name: kimi_k2_thinking.md
--------------------
title: (343) ChatGPT Atlas - The Battle for your Browser - YouTube
url: https://www.youtube.com/watch?v=KKdsSO1-LQE
file_name: openai_atlas_browser.md
--------------------
title: (343) AgentHQ by Github - YouTub

In [ ]:
# client.file_search_stores.upload_to_file_search_store()

In [ ]:
def upload_doc(file_path):
    title, url = extract_title_and_url(file_path )
    file_name = os.path.basename(file_path)
    # Import the file into the file search store with custom metadata
    operation = client.file_search_stores.upload_to_file_search_store(
        file_search_store_name=file_search_store.name,
        file = file_path,
        config={'display_name' : title,
                'chunking_config' : chunking_config["chunking_config"],
                'custom_metadata':[
                    {"key": "title", "string_value": title},
                    {"key": "url", "string_value": url},
                    {"key": "file_name", "string_value": file_name},
                    ],
                }
    )
    # Wait until import is complete
    while not operation.done:
        time.sleep(5)
        print("Waiting")
        operation = client.operations.get(operation)

    print(f"{file_name} is uploaded and indexed")

In [ ]:
#uploading the files
for file_path in transcript_files:
    upload_doc(file_path)

Waiting
deepseek_ocr.md is uploaded and indexed
Waiting
is_meta_killing_fair.md is uploaded and indexed
Waiting
langchain_1.0.md is uploaded and indexed
Waiting
claude_skills.md is uploaded and indexed
Waiting
kimi_k2_thinking.md is uploaded and indexed
Waiting
openai_atlas_browser.md is uploaded and indexed
Waiting
github_universe.md is uploaded and indexed
Waiting
haiku_4.5.md is uploaded and indexed


In [ ]:
# # Upload the file using the Files API, supply a file name which will be visible in citations
# sample_file = client.files.upload(file='sample.txt', config={'name': 'display_file_name'})

In [ ]:
PROMPT = "What is the name of the browser OpenAI Launched?"

#  Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}\n (return your answer in markdown as concise bullet points)\nANSWER:\n""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print(response.text)

*   OpenAI launched a browser named ChatGPT Atlas.


In [ ]:
print(response.candidates[0].grounding_metadata)

google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""(343) ChatGPT Atlas - The Battle for your Browser - YouTube
https://www.youtube.com/watch?v=KKdsSO1-LQE

Transcript:
(00:00) Okay, so OpenAI has just had their launch for ChatGPT Atlas, and this is basically their launch of a browser, or rather their re-skinning of the chromium browser from Google, they're not the first ones to do this. Earlier in the year, we had, perplexity do the same thing with Comet. and they basically launched a browser, with almost identical features, to what we've seen come out today from OpenAI.
(00:29) And I gotta be honest that if anything, I think perhaps the perplexity one is a little bit more interesting of where they're going and the kind of things that they're doing. But OpenAI is also following on from Anthropic, launching, as a Chrome extension, about a month and a half ago now, The key thing with all of these is that

In [ ]:

if response.candidates[0].grounding_metadata:
  for i, chunk in enumerate(response.candidates[0].grounding_metadata.grounding_chunks):
    print(f'--- Citation {i+1} ---')
    print(f'  - Retrieved context length: {len(chunk.retrieved_context.text)} characters')
    # Print the first 100 characters of the retrieved text for a preview.
    print(f'  - Preview: "{chunk.retrieved_context.text[:100]}..."\n')
else:
  print("No grounding metadata found in the response.")

--- Citation 1 ---
  - Retrieved context length: 1492 characters
  - Preview: "(343) ChatGPT Atlas - The Battle for your Browser - YouTube
https://www.youtube.com/watch?v=KKdsSO1-..."

--- Citation 2 ---
  - Retrieved context length: 1334 characters
  - Preview: "find out about the New James Bond movie. I'm curious to see like, what can it actually find? Where d..."

--- Citation 3 ---
  - Retrieved context length: 1376 characters
  - Preview: "a Google Chrome browser.
(01:14) So both perplexity and chatGPT have gone the way of, using chromium..."

--- Citation 4 ---
  - Retrieved context length: 1320 characters
  - Preview: "in here we can see, the actual, features Agent is one of the big things, that they're pushing in her..."

--- Citation 5 ---
  - Retrieved context length: 1353 characters
  - Preview: "enough to at least get people trying this out, so you can see here they're constantly trying to reas..."



In [ ]:
for key in response.candidates[0].grounding_metadata.__dict__.keys():
    print(key)

google_maps_widget_context_token
grounding_chunks
grounding_supports
retrieval_metadata
retrieval_queries
search_entry_point
source_flagging_uris
web_search_queries


In [ ]:
response.candidates[0].grounding_metadata.grounding_supports[0], response.candidates[0].grounding_metadata.grounding_chunks[0]

(GroundingSupport(
   grounding_chunk_indices=[
     0,
     1,
   ],
   segment=Segment(
     end_index=54,
     text='*   OpenAI has launched a browser named ChatGPT Atlas.'
   )
 ),
 GroundingChunk(
   retrieved_context=GroundingChunkRetrievedContext(
     text="""find out about the New James Bond movie. I'm curious to see like, what can it actually find? Where does it look? That kind of thing. don't forget, there are a lot of sites now that are blocking open ai, so how is that gonna work? If you're using the OpenAI browser, and you are trying to go to, some of these sites, are we gonna see that this browser is just not gonna work on certain sites, or not? Okay, we can see that it went off and did some stuff.
 (09:31) Now I didn't actually see where it went. so it's come back with information. the information I think is probably okay. We've got some nice, different citations. Really no different than what we would have in chat gpt.com. that is interesting. we can see, if we click he

In [ ]:
response.candidates[0].grounding_metadata.grounding_chunks[0]

GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""find out about the New James Bond movie. I'm curious to see like, what can it actually find? Where does it look? That kind of thing. don't forget, there are a lot of sites now that are blocking open ai, so how is that gonna work? If you're using the OpenAI browser, and you are trying to go to, some of these sites, are we gonna see that this browser is just not gonna work on certain sites, or not? Okay, we can see that it went off and did some stuff.
(09:31) Now I didn't actually see where it went. so it's come back with information. the information I think is probably okay. We've got some nice, different citations. Really no different than what we would have in chat gpt.com. that is interesting. we can see, if we click here, the different search results, we can see different images, really trying to basically just sort of become.
(09:56) I guess a search engine, in here. it is kind of funny that for videos,

In [ ]:
PROMPT = "What videos did Sam make about Claude? please give URLs"

# Use the metadata filter to search within a subset of documents
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                  file_search_store_names=[file_search_store.name],
                  )
            )
        ]
    )
)

print(response.text)

Sam has created the following videos related to Claude:

*   **Claude Skills - SOPs For Agents**: This video discusses how big companies are using agents to adopt standard operating procedures for various tasks, specifically mentioning Anthropic's "skills" feature for Claude.
    *   URL: `https://www.youtube.com/watch?v=fvUGQFtJaT4`
*   **Haiku 4.5 - Small Beats Big**: In this video, Sam examines Anthropic's Claude Haiku 4.5 model, discussing its pricing and its potential as a replacement for Sonnet 4, particularly for building agents.
    *   URL: `https://www.youtube.com/watch?v=lrf8eE-ADiw`


In [ ]:
PROMPT = "What does Sam talk about in this video?"

# Use the metadata filter to search within a subset of documents
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                  file_search_store_names=[file_search_store.name],
                  metadata_filter = 'title="(343) ChatGPT Atlas - The Battle for your Browser - YouTube"',
                  )
            )
        ]
    )
)

print(response.text)

In the video titled "(343) ChatGPT Atlas - The Battle for your Browser," the speaker discusses OpenAI's launch of ChatGPT Atlas, which is essentially a re-skinned Chromium browser. The video delves into the features of this new browser, including its agent mode, and provides a comparison with similar products from companies like Perplexity (with their browser, Comet) and Anthropic (with their Chrome extension).

The speaker emphasizes the emerging "battle for getting people to change their browser" and the strategic importance for companies in the "answer engine game" to have their own browser rather than relying on existing ones like Google Chrome. The video also touches upon the use of Chromium by both Perplexity and ChatGPT for their browsers, which allows them to leverage an existing platform rather than building one from scratch.

Furthermore, the video includes a demonstration of ChatGPT Atlas, specifically highlighting its agent mode and discussing its functionality, performance

In [ ]:
PROMPT = "What does Sam talk about in this video?" # https://www.youtube.com/watch?v=fvUGQFtJaT4

# Use the metadata filter to search within a subset of documents
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                  file_search_store_names=[file_search_store.name],
                  metadata_filter = 'url="https://www.youtube.com/watch?v=fvUGQFtJaT4"',
                  )
            )
        ]
    )
)

print(response.text)

In the video titled "(343) Claude Skills - SOPs For Agents," Sam discusses the importance of standard operating procedures (SOPs) for businesses, drawing from his past experience in manufacturing CDs, CDRs, and DVD-Rs where SOPs simplified operations.

He then transitions to how large companies are now enabling agents to adopt SOPs for various tasks. The main focus is on "skills," a feature recently announced by Anthropic. Sam also touches upon Claude Code plugins and Gemini CLI extensions, explaining that all three serve as methods to provide agents with SOPs to perform desired tasks.

Sam emphasizes that the quality of agents and the output from Large Language Models (LLMs) depend heavily on the context provided to them, much like humans need SOPs to reduce variability, agents require them even more. He describes this as a core pattern where inputs go through steps, decision points, tool usage, and then produce an output.

He further explains that packaging things as "skills" is key,

## Listing the docs in the vectore store

In [ ]:
#List Documents in a Vector Store
print(f"Listing Documents in Vector Store: {file_search_store.name}\n")
for document in client.file_search_stores.documents.list(parent=file_search_store.name):
  print(f' - Name: {document.name}')
  print(f'   - Display Name: {document.display_name}')

Listing Documents in Vector Store: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0

 - Name: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0/documents/343-deepseek-ocr-more-than--hp2aifdtpnku
   - Display Name: (343) DeepSeek OCR - More than OCR - YouTube
 - Name: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0/documents/343-is-meta-killing-fair-yo-3vdnylc3887u
   - Display Name: (343) Is Meta killing FAIR? - YouTube
 - Name: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0/documents/343-langchain-reaches-10-wh-0qu1zgytxdyc
   - Display Name: (343) LangChain Reaches 1.0 - Whats new? - YouTube
 - Name: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0/documents/343-claude-skills-sops-for--e4g0wdcnnq2t
   - Display Name: (343) Claude Skills - SOPs For Agents - YouTube
 - Name: fileSearchStores/samyoutubetranscripts-dr8xi4wp9gf0/documents/343-new-kimi-k2-thinking-th-e2a1h8gmg8ux
   - Display Name: (343) NEW Kimi K2 Thinking - Think Harder - YouTube
 - Name: fileSearchStor

### Deleting Your Stores

In [ ]:
# deleting the stores
# List all your file search stores
for file_search_store in client.file_search_stores.list():
    name = file_search_store.name
    print(name)
    # Get a specific file search store by name
    my_file_search_store = client.file_search_stores.get(name=file_search_store.name)

    # Delete a file search store
    client.file_search_stores.delete(name=name, config={'force': True})

fileSearchStores/samyoutubetranscripts-xudvlt38p3w5


In [ ]:
# Get a specific file search store by name
my_file_search_store = client.file_search_stores.get(name='fileSearchStores/my-file_search-store-123')

# Delete a file search store
client.file_search_stores.delete(name='fileSearchStores/my-file_search-store-123', config={'force': True})